In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

import os
import sys
import importlib
path_my_modules = "/home/born-again/Documents/GitHub/my_modules"
module_path = os.path.abspath(f'{path_my_modules}/config')
if module_path not in sys.path:
    sys.path.append(module_path)

import cfg
importlib.reload(cfg)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_utilities}')
if module_path not in sys.path:
    sys.path.append(module_path)

import utilities as utl
importlib.reload(utl)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_plot_style}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import plotter
importlib.reload(plotter)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_spectral_models}')
if module_path not in sys.path:
    sys.path.append(module_path)

import spectral_models as spec
importlib.reload(spec)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_lhaaso_analysis}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import lhaaso_analysis as lhaaso
importlib.reload(lhaaso)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_gammapy_catalogs}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import gammapy_catalogs as gammapy_cat
importlib.reload(gammapy_cat)

module_path = os.path.abspath(f'{path_my_modules}/{cfg.dir_cta_simulation}')
if module_path not in sys.path:
    print(sys.path.append(module_path))
    sys.path.append(module_path)

import cta_simulation as cta
importlib.reload(cta)

%matplotlib inline
import matplotlib.pyplot as plt # A collection of command style functions


# path_my_plot_style = f"{path_my_modules}/{cfg.dir_plot_style}/my_plot_style.txt" 
# plt.style.use(path_my_plot_style)

In [ ]:
from astropy import units as u
from gammapy.datasets import Datasets
from gammapy.modeling import Fit
from gammapy.modeling.models import Models, NaimaSpectralModel, SkyModel
import naima
from astropy.io import ascii

In [ ]:
datasets_name = 'CTA-PSR_J1826-1334'
path_file = f"/home/born-again/Documents/GitHub/JCAP_095P_1123/my_notebooks/pulsars/analysis/datasets/LHAASO_J1825-1326_roi_1.0deg_e_ref_min_100.0GeV/HESS_J1825-137-CTA-PSR_J1826-1334_50h"
datasets_joint =Datasets.read(filename=f"{path_file}/datasets{cfg.format_yaml}", filename_models=f"{path_file}/models{cfg.format_yaml}")
models_joint = Models()
models_joint.extend(datasets_joint.models)
dict_leg_style = utl.load_dictionary(name=cfg.dict_leg_style, path_file=path_file)
dict_sep = utl.load_dictionary(name=cfg.dict_separation, path_file=path_file)

In [ ]:
print(datasets_joint[-1])

In [ ]:
datasets = Datasets(datasets_joint[-1].copy())

In [ ]:
data = datasets[0].data.to_table(sed_type="dnde")

In [ ]:
data

https://naima.readthedocs.io/en/latest/dataformat.html#dataformat

In [ ]:
data.rename_column('e_ref', 'energy')
data.rename_column('e_min', 'energy_lo')
data.rename_column('e_max', 'energy_hi')
data.rename_column('dnde', 'flux')
data.rename_column('dnde_err', 'flux_error')
data.rename_column('dnde_ul', 'flux_ul')
data.rename_column('is_ul', 'ul')


In [ ]:
data = data[["energy","energy_lo","energy_hi","flux","flux_error", "flux_ul", "ul"]]

In [ ]:
data

In [ ]:
from naima.models import ExponentialCutoffPowerLaw, InverseCompton
import astropy.units as u


def model(pars, data):
    amplitude = pars[0] / u.eV
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV

    ECPL = ExponentialCutoffPowerLaw(amplitude, 10 * u.TeV, alpha, e_cutoff)
    IC = InverseCompton(
        ECPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm ** 3],
        ],
    )

    return IC.flux(data, distance=3.606 * u.kpc)

In [ ]:
from naima import uniform_prior


def lnprior(pars):
    lnprior = uniform_prior(pars[0], 0.0, np.inf) + uniform_prior(pars[1], -1, 5) + uniform_prior(pars[2], 0.0, 2)
    return lnprior

In [ ]:
import numpy as np

p0 = np.array((1e33, 3.0, np.log10(30)))
labels = ["norm", "index", "log10(cutoff)"]

In [ ]:
sampler, pos = naima.run_sampler(
    data_table=data,
    p0=p0,
    labels=labels,
    model=model,
    prior=lnprior,
    nwalkers=128,
    nburn=50,
    nrun=10,
    threads=4,
)

In [ ]:
# import naima
# import astropy.units as u
# import matplotlib.pyplot as plt

In [ ]:
naima.save_diagnostic_plots(
    datasets_name,
    sampler,
    blob_labels=[
        "Spectrum",
        "Electron energy distribution",
        "$W_e (E_e>1$ TeV)",
    ],
)

naima.save_results_table(datasets_name, sampler)

In [ ]:
 naima.plot_fit(sampler, modelidx=0, label="sss", sed=True, last_step=False, n_samples=100, confs=[3,1], ML_info=False, figure=None, plotdata=None, plotresiduals=None, e_unit=u.TeV,e_range=[100*u.GeV, 79*u.TeV], e_npoints=100, threads=None, xlabel=None, ylabel=None, ulim_opts={}, errorbar_opts={})